In [1]:
import threading
from queue import Queue

In [2]:
threading.active_count()   # 获取已激活的线程数

5

In [3]:
threading.enumerate()    # 查看所有线程信息

[<_MainThread(MainThread, started 16132)>,
 <Thread(Thread-4, started daemon 1344)>,
 <Heartbeat(Thread-5, started daemon 2180)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 14240)>,
 <ParentPollerWindows(Thread-3, started daemon 16684)>]

In [4]:
threading.current_thread()  # 查看正在运行的线程

<_MainThread(MainThread, started 16132)>

In [5]:
def thread_job():
    print()

SyntaxError: invalid syntax (<ipython-input-5-01d2f6776926>, line 1)